In [13]:
import random, re, csv
import numpy as np
import MLData,MLMethod
#for features and labels
from sklearn.model_selection import train_test_split
#for analysis methods
from sklearn import svm
#for evaluation
from scipy import stats
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.model_selection import cross_val_score

#======================================
# Get data for analysis
#======================================
# Get Raw Data From Dataset
#ilpddatafiledir = '/afs/inf.ed.ac.uk/user/s14/s1458248/workspace/vinlccanalysissystem/src/main/resources/ilpddata/ilpddata2.csv'
ilpddatafiledir = '/Users/cancui/workspace/virENV/lccanalysissystem/src/main/resources/ilpddata/ilpddata2.csv'
ilpddata =MLData.get_ilpddata(ilpddatafiledir)
biodata_patterns=[]
for i in ilpddata:
    biodata_patterns=biodata_patterns+[i[0:10]]
interdata_patterns= MLData.get_interdata_patterns_sim(ilpddata)

#=======================================
# Interaction data analysis
#=======================================    
num_clusters=2
biodata=MLMethod.label_biodata(num_clusters,interdata_patterns,biodata_patterns)


In [14]:
#==========================================================
# bio information classification
#==========================================================

#============================ SVM ================================
ml_biodata = MLData.get_features_labels(biodata,10,0.4)
svm_bio_train_features=ml_biodata['train_features'][:,0:10]
svm_bio_train_labels=ml_biodata['train_labels']
svm_bio_test_features=ml_biodata['test_features'][:,0:10]
svm_bio_test_labels=ml_biodata['test_labels']

svmclf=svm.SVC(C=1)
svmclf.fit(svm_bio_train_features,svm_bio_train_labels)
svmclfresult=svmclf.predict(svm_bio_test_features)

scores=cross_val_score(svmclf,svm_bio_train_features,svm_bio_train_labels,cv=4)

'''
print scores
print accuracy_score(svm_bio_test_labels,svmclfresult)
'''


'\nprint scores\nprint accuracy_score(svm_bio_test_labels,svmclfresult)\n'

In [20]:
#========================== NN ===================================
from sklearn import datasets, svm, linear_model
import tensorflow as tf
import numpy as np
import MLData

# DATA
nn_bio_train_features=ml_biodata['train_features']
nn_bio_test_features=ml_biodata['test_features']
nn_bio_train_labels=[]
nn_bio_test_labels=[]

for i in ml_biodata['train_labels']:
    if i==0:
        nn_bio_train_labels=nn_bio_train_labels+[[0,1]]
    else:
        nn_bio_train_labels=nn_bio_train_labels+[[1,0]]

for i in ml_biodata['test_labels']:
    if i==0:
        nn_bio_test_labels=nn_bio_test_labels+[[0,1]]
    else:
        nn_bio_test_labels=nn_bio_test_labels+[[1,0]]

nn_bio_train_labels=np.array(nn_bio_train_labels)
nn_bio_test_labels=np.array(nn_bio_test_labels)


In [21]:
# Set Network Structure
## variables for network
num_features=10
num_hidden_units_layer1=10
num_output_units=2

## input/output information
NN_input=tf.placeholder(tf.float32,[None, num_features],name="NN_input")
NN_output=tf.placeholder(tf.float32,[None, num_output_units],name="NN_output")

## hidden layer1
weight1=tf.get_variable('w1',[num_features,num_hidden_units_layer1],tf.float32)
biase1=tf.get_variable('b1',[num_hidden_units_layer1],tf.float32)
l1_output=tf.sigmoid(tf.matmul(NN_input,weight1)+biase1)

## output layer
weight2=tf.get_variable('w2',[num_hidden_units_layer1,num_output_units],tf.float32)
biase2=tf.get_variable('b2',[num_output_units],tf.float32)
output_logits=tf.matmul(l1_output,weight2)+biase2

In [22]:
## train methods
### loss function
cross_entropy=tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=NN_output,logits=output_logits))
l1_regularizer=tf.contrib.layers.l1_regularizer(scale=0.1,scope=None)
loss_fun=cross_entropy+tf.contrib.layers.apply_regularization(l1_regularizer,[weight1])
### train method
train_step=tf.train.GradientDescentOptimizer(0.05).minimize(loss_fun)

In [23]:
# Train Network
sess=tf.InteractiveSession()
tf.global_variables_initializer().run()
for i in range(1000):
    batch_xs,batch_ys=(nn_bio_train_features,nn_bio_train_labels)
    sess.run(train_step,feed_dict={NN_input:batch_xs,NN_output:batch_ys})

In [24]:
# Evaluation
## test trained model
correct_prediction=tf.equal(tf.argmax(NN_output,1),tf.argmax(output_logits,1))
prediction_accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

train_accuracy=sess.run(prediction_accuracy,feed_dict={NN_input:nn_bio_train_features,NN_output:nn_bio_train_labels})
test_accuracy=sess.run(prediction_accuracy,feed_dict={NN_input:nn_bio_test_features,NN_output:nn_bio_test_labels})


# Show variable
w1=sess.run(weight1)
weight1_sum=np.sum(w1,axis=1)
min_feature=np.argmin(np.abs(weight1_sum))


print test_accuracy
print min_feature

0.706897
2
